We can also calculate MVRV results and save them in a local SQLite database, which offers better scalability and efficiency compared to storing data in CSV files, especially as the dataset grows over time. You can also automates hourly updates using the *schedule* library to continuously append new data points, There is also a helper function included to query recent records from the database for analysis. The code below is a demo for Bitcoin but the same can be done for Solana by replacing the data fetching and calculating functions from the ones used in the *Solana MVRV Calculation* notebook.

**Data Fetching and Calculation Functions:**
- *get_btc_supply()* :
Fetches Bitcoin's circulating supply from the CoinGecko API
- *get_price_volume_data()* :
Retrieves the last 365 days of Bitcoin price and volume data from the CoinGecko market chart API. It converts the returned timestamps to datetime objects and merges price and volume data into a single DataFrame.
- *get_current_btc_price()* :
Gets the current Bitcoin price in USD from CoinGecko’s simple price API.

- *calculate_mvrv()* :
Calls the above data-fetching functions to get price-volume data, circulating supply, and current price. It calculates VWAP, Realized Value (RV), Market Value(MV) and MVRV ratio. It prints and returns these key metrics along with a timestamp.

**Database Storage Functions:**

- *save_to_db(data, db_path="mvrv_data.db")* :
Connects to a local SQLite database (or creates one if missing), ensures the mvrv_history table exists, and inserts the latest MVRV data into it, then closes the connection.

- *read_mvrv_data(db_path="mvrv_data.db", limit=5)* :
Queries the SQLite database to read the most recent limit entries from the mvrv_history table, returning the results as a pandas DataFrame.

**Scheduling and Execution:**
The script installs and uses the schedule library to set up a job that runs the MVRV calculation and saves it every hour automatically. I am using it to demo for 3 hours but you can keep it running inside an infinite loop.

In [ ]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime

# Fetch Circulating Supply
def get_btc_supply():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin"
    params = {
        "localization": "false",
        "tickers": "false",
        "market_data": "true",
        "community_data": "false",
        "developer_data": "false",
        "sparkline": "false"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        return data["market_data"]["circulating_supply"]
    except Exception as e:
        print("Error fetching BTC supply:", e)
        return None

# Fetch 365 Days of Price/Volume
def get_price_volume_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {"vs_currency": "usd", "days": "365"}
    try:
        response = requests.get(url, params=params)
        data = response.json()

        # Convert to DataFrame
        prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
        volumes = pd.DataFrame(data["total_volumes"], columns=["timestamp", "volume"])

        prices["date"] = pd.to_datetime(prices["timestamp"], unit="ms")
        volumes["date"] = pd.to_datetime(volumes["timestamp"], unit="ms")

        df = pd.merge(prices[["date", "price"]], volumes[["date", "volume"]], on="date")
        return df

    except Exception as e:
        print("Error fetching price/volume data:", e)
        return None

# Fetch Current BTC Price
def get_current_btc_price():
    url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
    try:
        response = requests.get(url)
        return response.json()['bitcoin']['usd']
    except Exception as e:
        print("Error fetching current price:", e)
        return None

# Calculate MVRV Ratio
def calculate_mvrv():
    df = get_price_volume_data()
    btc_supply = get_btc_supply()
    current_price = get_current_btc_price()

    if df is None or btc_supply is None or current_price is None:
        print("Missing data. Cannot compute MVRV.")
        return

    # Calculate VWAP
    vwap = (df["price"] * df["volume"]).sum() / df["volume"].sum()
    realized_value = vwap * btc_supply
    market_value = current_price * btc_supply
    mvrv = market_value / realized_value

    # Print results
    print(f"Current BTC Price: ${current_price:,.2f}")
    print(f"Circulating BTC Supply: {btc_supply:,.0f}")
    print(f"VWAP (approx realized price): ${vwap:,.2f}")
    print(f"Realized Value: ${realized_value:,.2f}")
    print(f"Market Value: ${market_value:,.2f}")
    print(f"MVRV Ratio: {mvrv:.4f}")

    # Optionally return all values
    return {
        "timestamp": datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'),
        "btc_price": current_price,
        "btc_supply": btc_supply,
        "vwap": vwap,
        "realized_value": realized_value,
        "market_value": market_value,
        "mvrv": mvrv
    }

#  Save to SQLite
def save_to_db(data, db_path="mvrv_data.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mvrv_history (
            timestamp TEXT,
            btc_price REAL,
            btc_supply REAL,
            vwap REAL,
            realized_value REAL,
            market_value REAL,
            mvrv REAL
        )
    ''')
    cursor.execute('''
        INSERT INTO mvrv_history VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        data['timestamp'],
        data['btc_price'],
        data['btc_supply'],
        data['vwap'],
        data['realized_value'],
        data['market_value'],
        data['mvrv']
    ))
    conn.commit()
    conn.close()
    print("Saved to database")

# Run all
if __name__ == "__main__":
    result = calculate_mvrv()
    if result:
        save_to_db(result)

Current BTC Price: $107,383.00
Circulating BTC Supply: 19,884,043
VWAP (approx realized price): $86,587.49
Realized Value: $1,721,709,323,474.73
Market Value: $2,135,208,189,469.00
MVRV Ratio: 1.2402
Saved to database


In [ ]:
! pip install schedule

In [ ]:
import schedule
import time

def job():
    result = calculate_mvrv()
    if result:
        save_to_db(result)
    print("Job done.")

schedule.every().hour.do(job)

start_time = time.time()
run_duration_seconds = 3 * 60 * 60  # 3 hours in seconds

while True:
    schedule.run_pending()
    time.sleep(1)

    elapsed_time = time.time() - start_time
    if elapsed_time > run_duration_seconds:
        print("Run duration reached 3 hours. Stopping scheduler.")
        break

Current BTC Price: $107,357.00
Circulating BTC Supply: 19,884,043
VWAP (approx realized price): $86,589.73
Realized Value: $1,721,753,849,268.22
Market Value: $2,134,691,204,351.00
MVRV Ratio: 1.2398
Saved to database
Job done.
Current BTC Price: $107,240.00
Circulating BTC Supply: 19,884,043
VWAP (approx realized price): $86,590.50
Realized Value: $1,721,769,190,969.24
Market Value: $2,132,364,771,320.00
MVRV Ratio: 1.2385
Saved to database
Job done.
Run duration reached 3 hours. Stopping scheduler.


To print the most recent 5 records from the database.

In [ ]:
def read_mvrv_data(db_path="mvrv_data.db", limit=5):
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM mvrv_history ORDER BY timestamp DESC LIMIT {limit}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

df_sample = read_mvrv_data()
print(df_sample)

             timestamp  btc_price  btc_supply          vwap  realized_value  \
0  2025-06-27 07:39:45   107240.0  19884043.0  86590.498269    1.721769e+12   
1  2025-06-27 06:39:44   107357.0  19884043.0  86589.726710    1.721754e+12   
2  2025-06-27 05:39:25   107383.0  19884043.0  86587.487438    1.721709e+12   

   market_value      mvrv  
0  2.132365e+12  1.238473  
1  2.134691e+12  1.239835  
2  2.135208e+12  1.240168  


To get daily_mvrv calculated by aggregating hourly data.

In [ ]:
def read_daily_mvrv(db_path="mvrv_data.db", limit=7):
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT
            DATE(timestamp) AS date,
            AVG(btc_price) AS avg_btc_price,
            AVG(vwap) AS avg_vwap,
            AVG(realized_value) AS avg_realized_value,
            AVG(market_value) AS avg_market_value,
            AVG(mvrv) AS avg_mvrv
        FROM mvrv_history
        GROUP BY DATE(timestamp)
        ORDER BY DATE(timestamp) DESC
        LIMIT {limit}
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

daily_df = read_daily_mvrv()
print(daily_df)
